In [1]:
import numpy as np
import pandas as pd
from pandas.tseries.offsets import Hour, Minute
from pandas.tseries.offsets import Day, MonthEnd
from pandas.tseries.offsets import Hour
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error as mse
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

from tensorflow.keras.utils import set_random_seed as set_seed
set_seed(42)


In [2]:


#df = pd.read_csv('Wind_deseason.csv')
df = pd.read_csv('PV_deseason_15.csv', parse_dates=[0], index_col=0)


In [3]:
def TrainTest(df):
    
    num_rows = df.shape[0]
    
    num_training = (num_rows * 8)//10
   
    #train = df.iloc[0:num_training]
    #test = df.iloc[num_training:num_rows]
    
    
    return num_training 
        
            

In [4]:
def lags(df, n):
    
    df_new = df
    
    for i in range(n):
        name = "lag "+ str(i+1)
        df_new[name] = df['310_PV_1'].shift(i+1)
    
    return df_new

In [5]:

def model1Train(X,Y):
     
    mod = Sequential()
    mod.add(LSTM(6, activation='sigmoid', input_shape=(107, 1), return_sequences = True))
    mod.add(LSTM(6, activation='sigmoid', input_shape=(107, 1), return_sequences = True))
    mod.add(LSTM(6, activation='sigmoid', input_shape=(107, 1)))
    mod.add(Dense(1))
    mod.compile(optimizer='adam', loss='mse', metrics = ['accuracy'])
       
        
    print(mod.summary())
        
    mod.fit(x = X, y = Y,epochs=20,verbose = 1)#keep low for now, edit later
    
    return mod
    
    
    

In [6]:
def model2Train(gen):
     
    mod = Sequential()
    mod.add(LSTM(1, activation='sigmoid', input_shape=(107, 1)))
   
    mod.add(Dense(1))
    mod.compile(optimizer='adam', loss='mse', metrics = ['accuracy'])
       
        
    print(mod.summary())
        
    mod.fit(gen,epochs=1,verbose = 1)#keep low for now, edit later
    
    return mod
    

In [7]:
def TestModel(X,test, mod, scaler):
    
    MSEs = []
    
    for i in range(len(X)):
        array = np.array(X[i])
        
        pred = mod.predict(array.reshape(1,107,1))
        print(pred)
        #pred = scaler.inverse_transform(pred)
        unscaled_pred = scaler.inverse_transform(pred)
        unscaled_test = scaler.inverse_transform([[test[i]]])
       
        
        MSEs.append(mse(unscaled_test[0],unscaled_pred[0]))
    return MSEs

In [8]:
#def modelNaiveTest(df):

    
    
    


In [9]:
def runModels(df, last):
    
    scaler = MinMaxScaler()
    
    bound = TrainTest(df)
    
    train = df[0:bound]
    test = df[bound:]
    
    
    scaler.fit(train['310_PV_1'].to_frame())
    
    scaled_df = scaler.transform(df['310_PV_1'].to_frame())
    scaled_last = scaler.transform(last['310_PV_1'].to_frame())
    scaled_df = np.append(scaled_last, scaled_df)
    
    generator1 =  TimeseriesGenerator(scaled_df, scaled_df, length=107, batch_size=1)
    generator = TimeseriesGenerator(scaled_df[107:bound+107], scaled_df[107:bound+107], length=107, batch_size=1)
    
    
    scaled_df2 = pd.DataFrame(scaled_df, columns = ['310_PV_1'])
    
    lag = lags(scaled_df2, 107)
    
    lag = lag[lag.columns[::-1]]
    
    lagged_train = lag.iloc[107:bound+107]
    lagged_test = lag.iloc[bound+107:]

    
    
    X_test = lagged_test.drop('310_PV_1', axis = 1).values.tolist()
    Y_test = lagged_test['310_PV_1'].values.tolist()
  
    
    model2 = model2Train(generator)
    
    Tests = TestModel(X_test, Y_test, model2, scaler)
    
    
    print(Tests)
    
   
    
    return Tests#, mse2

    

In [10]:


df1 = df[df.index.month.isin([1,2,3])]
df1 = df1.iloc[:-1 , :]


df2 = df[df.index.month.isin([4,5,6])]

df3 = df[df.index.month.isin([7,8,9])]

df4 = df[df.index.month.isin([10,11,12])]

In [11]:
 #mse1 = runModels(df1,df4.iloc[-107:])


In [12]:
#mse3 = runModels(df2,df1.iloc[-107:])

In [13]:
#mse5 = runModels(df3, df2.iloc[-107:])

In [14]:
mse7 = runModels(df4, df3.iloc[-107:])

C:\Users\bvits\AppData\Local\Temp\ipykernel_21088\610209718.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_new[name] = df['310_PV_1'].shift(i+1)
C:\Users\bvits\AppData\Local\Temp\ipykernel_21088\610209718.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_new[name] = df['310_PV_1'].shift(i+1)
C:\Users\bvits\AppData\Local\Temp\ipykernel_21088\610209718.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joinin

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 1)                 12        
                                                                 
 dense (Dense)               (None, 1)                 2         
                                                                 
Total params: 14
Trainable params: 14
Non-trainable params: 0
_________________________________________________________________
None
1/1 [==============================] - 0s 204ms/step
[[0.82363427]]
1/1 [==============================] - 0s 22ms/step
[[0.6738708]]
1/1 [==============================] - 0s 22ms/step
[[0.69082665]]
1/1 [==============================] - 0s 21ms/step
[[0.6858134]]
1/1 [==============================] - 0s 20ms/step
[[0.6922535]]
1/1 [==============================] - 0s 24ms/step
[[0.7930027]]
1/1 [==============================] - 0s 24ms/step


[[0.658183]]
1/1 [==============================] - 0s 23ms/step
[[0.68417346]]
1/1 [==============================] - 0s 24ms/step
[[0.7115003]]
1/1 [==============================] - 0s 22ms/step
[[0.73364747]]
1/1 [==============================] - 0s 22ms/step
[[0.74490225]]
1/1 [==============================] - 0s 24ms/step
[[0.7507651]]
1/1 [==============================] - 0s 22ms/step
[[0.75449514]]
1/1 [==============================] - 0s 22ms/step
[[0.7574812]]
1/1 [==============================] - 0s 21ms/step
[[0.7598975]]
1/1 [==============================] - 0s 21ms/step
[[0.76185966]]
1/1 [==============================] - 0s 19ms/step
[[0.763458]]
1/1 [==============================] - 0s 20ms/step
[[0.7647637]]
1/1 [==============================] - 0s 22ms/step
[[0.7658324]]
1/1 [==============================] - 0s 20ms/step
[[0.7667091]]
1/1 [==============================] - 0s 22ms/step
[[0.76742935]]
1/1 [==============================] - 0s 21ms/step
[[0.76

1/1 [==============================] - 0s 21ms/step
[[0.77120566]]
1/1 [==============================] - 0s 21ms/step
[[0.77125525]]
1/1 [==============================] - 0s 43ms/step
[[0.7712971]]
1/1 [==============================] - 0s 21ms/step
[[0.771333]]
1/1 [==============================] - 0s 21ms/step
[[0.77136374]]
1/1 [==============================] - 0s 20ms/step
[[0.7713902]]
1/1 [==============================] - 0s 33ms/step
[[0.7714131]]
1/1 [==============================] - 0s 25ms/step
[[0.77143323]]
1/1 [==============================] - 0s 21ms/step
[[0.77145076]]
1/1 [==============================] - 0s 22ms/step
[[0.7714665]]
1/1 [==============================] - 0s 22ms/step
[[0.77148056]]
1/1 [==============================] - 0s 22ms/step
[[0.7714932]]
1/1 [==============================] - 0s 21ms/step
[[0.77150476]]
1/1 [==============================] - 0s 22ms/step
[[0.7715155]]
1/1 [==============================] - 0s 21ms/step
[[0.7715254]]
1/1 

1/1 [==============================] - 0s 23ms/step
[[0.5161941]]
1/1 [==============================] - 0s 25ms/step
[[0.58977556]]
1/1 [==============================] - 0s 35ms/step
[[0.64459753]]
1/1 [==============================] - 0s 20ms/step
[[0.6802547]]
1/1 [==============================] - 0s 42ms/step
[[0.6934601]]
1/1 [==============================] - 0s 21ms/step
[[0.7033737]]
1/1 [==============================] - 0s 22ms/step
[[0.7200239]]
1/1 [==============================] - 0s 21ms/step
[[0.7319062]]
1/1 [==============================] - 0s 21ms/step
[[0.73921907]]
1/1 [==============================] - 0s 21ms/step
[[0.7334957]]
1/1 [==============================] - 0s 23ms/step
[[0.73545194]]
1/1 [==============================] - 0s 21ms/step
[[0.7073009]]
1/1 [==============================] - 0s 20ms/step
[[0.700866]]
1/1 [==============================] - 0s 22ms/step
[[0.715757]]
1/1 [==============================] - 0s 21ms/step
[[0.6855092]]
1/1 [===

...

1/1 [==============================] - 0s 24ms/step
[[0.6342305]]
1/1 [==============================] - 0s 20ms/step
[[0.5907773]]
1/1 [==============================] - 0s 21ms/step
[[0.5402746]]
1/1 [==============================] - 0s 28ms/step
[[0.49605748]]
1/1 [==============================] - 0s 22ms/step
[[0.5705904]]
1/1 [==============================] - 0s 19ms/step
[[0.6395538]]
1/1 [==============================] - 0s 21ms/step
[[0.6877022]]
1/1 [==============================] - 0s 21ms/step
[[0.70923436]]
1/1 [==============================] - 0s 23ms/step
[[0.7307359]]
1/1 [==============================] - 0s 19ms/step
[[0.7577591]]
1/1 [==============================] - 0s 23ms/step
[[0.78054154]]
1/1 [==============================] - 0s 20ms/step
[[0.786582]]
1/1 [==============================] - 0s 49ms/step
[[0.7836549]]
1/1 [==============================] - 0s 23ms/step
[[0.78336704]]
1/1 [==============================] - 0s 22ms/step
[[0.7896544]]
1/1 [==

1/1 [==============================] - 0s 22ms/step
[[0.71504736]]
1/1 [==============================] - 0s 20ms/step
[[0.70675707]]
1/1 [==============================] - 0s 21ms/step
[[0.70471454]]
1/1 [==============================] - 0s 21ms/step
[[0.70304847]]
1/1 [==============================] - 0s 23ms/step
[[0.7049383]]
1/1 [==============================] - 0s 21ms/step
[[0.7061242]]
1/1 [==============================] - 0s 51ms/step
[[0.7077459]]
1/1 [==============================] - 0s 53ms/step
[[0.70216036]]
1/1 [==============================] - 0s 20ms/step
[[0.6904378]]
1/1 [==============================] - 0s 18ms/step
[[0.67372]]
1/1 [==============================] - 0s 19ms/step
[[0.64768493]]
1/1 [==============================] - 0s 22ms/step
[[0.6060364]]
1/1 [==============================] - 0s 22ms/step
[[0.5328611]]
1/1 [==============================] - 0s 22ms/step
[[0.5461749]]
1/1 [==============================] - 0s 19ms/step
[[0.5745506]]
1/1 [=

1/1 [==============================] - 0s 20ms/step
[[0.7731786]]
1/1 [==============================] - 0s 21ms/step
[[0.7742845]]
1/1 [==============================] - 0s 21ms/step
[[0.775193]]
1/1 [==============================] - 0s 20ms/step
[[0.77594054]]
1/1 [==============================] - 0s 21ms/step
[[0.7765565]]
1/1 [==============================] - 0s 22ms/step
[[0.77706504]]
1/1 [==============================] - 0s 22ms/step
[[0.7774854]]
1/1 [==============================] - 0s 24ms/step
[[0.777833]]
1/1 [==============================] - 0s 25ms/step
[[0.778121]]
1/1 [==============================] - 0s 20ms/step
[[0.77836]]
1/1 [==============================] - 0s 23ms/step
[[0.7785585]]
1/1 [==============================] - 0s 23ms/step
[[0.7787237]]
1/1 [==============================] - 0s 21ms/step
[[0.7788613]]
1/1 [==============================] - 0s 28ms/step
[[0.7789761]]
1/1 [==============================] - 0s 22ms/step
[[0.77907205]]
1/1 [=======

1/1 [==============================] - 0s 22ms/step
[[0.7801858]]
1/1 [==============================] - 0s 20ms/step
[[0.78020096]]
1/1 [==============================] - 0s 23ms/step
[[0.7802144]]
1/1 [==============================] - 0s 20ms/step
[[0.78022695]]
1/1 [==============================] - 0s 21ms/step
[[0.7802383]]
1/1 [==============================] - 0s 21ms/step
[[0.78024864]]
1/1 [==============================] - 0s 23ms/step
[[0.78025854]]
1/1 [==============================] - 0s 22ms/step
[[0.7802676]]
1/1 [==============================] - 0s 41ms/step
[[0.7802763]]
1/1 [==============================] - 0s 22ms/step
[[0.77730644]]
1/1 [==============================] - 0s 21ms/step
[[0.7647451]]
1/1 [==============================] - 0s 21ms/step
[[0.75240946]]
1/1 [==============================] - 0s 21ms/step
[[0.7266121]]
1/1 [==============================] - 0s 22ms/step
[[0.69319046]]
1/1 [==============================] - 0s 21ms/step
[[0.66267884]]
1/

1/1 [==============================] - 0s 45ms/step
[[0.768916]]
1/1 [==============================] - 0s 21ms/step
[[0.7645712]]
1/1 [==============================] - 0s 21ms/step
[[0.77044547]]
1/1 [==============================] - 0s 52ms/step
[[0.7681596]]
1/1 [==============================] - 0s 20ms/step
[[0.77119625]]
1/1 [==============================] - 0s 22ms/step
[[0.7701024]]
1/1 [==============================] - 0s 20ms/step
[[0.7640823]]
1/1 [==============================] - 0s 21ms/step
[[0.76329505]]
1/1 [==============================] - 0s 23ms/step
[[0.7535634]]
1/1 [==============================] - 0s 20ms/step
[[0.748507]]
1/1 [==============================] - 0s 23ms/step
[[0.7599561]]
1/1 [==============================] - 0s 21ms/step
[[0.7597965]]
1/1 [==============================] - 0s 19ms/step
[[0.76807594]]
1/1 [==============================] - 0s 21ms/step
[[0.7629787]]
1/1 [==============================] - 0s 22ms/step
[[0.7643008]]
1/1 [===

In [15]:
#print('season 1')

#print('model 1:')
#print(mse1)
#print('model Naive:')
#print(mse2)


In [16]:
#print('season 1')

#print('model 1:')
#print(mse3)
#print('model Naive:')
#print(mse4)


In [17]:
#print('season 1')

#print('model 1:')
#print(mse5)
#print('model Naive:')
#print(mse6)


In [18]:
#print('season 1')

#print('model 1:')
#print(mse7)
#print('model Naive:')
#print(mse8)


In [19]:

d = {'Season4':mse7}
df_errors_LSTM = pd.DataFrame({k:pd.Series(v) for k,v in d.items()})
df_errors_LSTM.to_csv('LSTM_errors_scaled_season4.csv')
#'Season1':mse1,'Season2':mse3,'Season3':mse5,
